In [1]:
import torch

_ = torch.tensor([0.2126, 0.7152, 0.0722], names=["c"])

/var/folders/0t/6d51btm11zz6lw61fgrmf7z00000gn/T/ipykernel_5709/2793260909.py:3: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/c10/core/TensorImpl.h:1938.)
  _ = torch.tensor([0.2126, 0.7152, 0.0722], names=["c"])


In [2]:
img_t = torch.randn(3, 5, 5)
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [ ]:
batch_t = torch.randn(2, 3, 5, 5)  # shape [batch, channels, rows, columns]

In [5]:
# 最後から3番目の次元（=チャンネル次元）で平均を取り、RGB画像をグレースケール化する
img_gray_naive = img_t.mean(-3)  # img_t: 単一画像（例：形状[3,H,W]または[1,3,H,W]）
batch_gray_naive = batch_t.mean(-3)  # batch_t: バッチ画像（例：形状[N,3,H,W]）

# グレースケール化されたテンソルの形状を確認
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [6]:
# 重みベクトル（例：RGBチャンネル用 [0.2989, 0.5870, 0.1140]）をテンソルに変換し、
# 高さ・幅方向にブロードキャスト可能なように、2回unsqueezeして形状を [3,1,1] にする
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)  # shape: [3,1,1]

# 各画素に対して、チャンネルごとに重みを掛ける（RGBごとの重み付き画像）
img_weights = img_t * unsqueezed_weights  # shape: [3,H,W]
batch_weights = batch_t * unsqueezed_weights  # shape: [N,3,H,W]

# チャンネル方向（最後から3番目）に沿って重み付き和を取り、グレースケール化
img_gray_weighted = img_weights.sum(-3)  # shape: [H,W]
batch_gray_weighted = batch_weights.sum(-3)  # shape: [N,H,W]

# 各テンソルの最終的な形状を確認
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [7]:
# einsum（Einstein summation）を使って、RGB画像にチャンネル重みをかけてグレースケール化
# '...chw,c->...hw' の意味：
#   - img_t や batch_t は (..., C, H, W) 形状をしており、C（チャンネル）と重み c を掛け合わせて和を取る
#   - ... はバッチ次元や他の任意の次元を保持する
#   - 出力はチャンネルを消して (..., H, W) になる

img_gray_weighted_fancy = torch.einsum(
    "...chw,c->...hw", img_t, weights
)  # shape: [H, W]
batch_gray_weighted_fancy = torch.einsum(
    "...chw,c->...hw", batch_t, weights
)  # shape: [N, H, W]

# バッチ版グレースケール画像の形状を確認
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [8]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=["channels"])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [9]:
# テンソルに名前付き次元（named tensor）を導入する
# refine_names(..., 'channels', 'rows', 'columns') は、最後の3次元をそれぞれ 'channels', 'rows', 'columns' と名付ける
# '...' は残りの先頭次元（例: バッチ次元など）を維持するという意味

img_named = img_t.refine_names(..., "channels", "rows", "columns")
batch_named = batch_t.refine_names(..., "channels", "rows", "columns")

# 名前付きテンソルの形状と次元名を確認
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [12]:
# weights_named は名前付きテンソル（例: names=['channels']）であると仮定
# img_named の名前付き次元 ['channels', 'rows', 'columns'] に合わせて、
# weights_named の次元を並び替え＆ブロードキャスト可能な形に調整する

weights_aligned = weights_named.align_as(img_named)

# weights_aligned の形状と名前付き次元を確認
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [13]:
gray_named = (img_named * weights_aligned).sum("channels")
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))